# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных:

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Условия задачи:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

<div class='alert alert-success'> ✔️Хорошо, что есть описание, по прошествии времени оно поможет сориентироваться в работе и восстановить схему своих действий.



</div>

## Загрузка и подготовка данных

In [12]:
import pandas as pd
import numpy as np
from scipy import stats
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [13]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Посмотрим на первые строки и общую информацию о данных:

In [14]:
display(data_0.head())
data_0.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [15]:
display(data_1.head())
data_1.info()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [16]:
display(data_2.head())
data_2.info()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Удалим столбец 'id', так как он не является информативным для обучения модели.

In [17]:
data_0.drop('id', axis=1, inplace=True)
data_1.drop('id', axis=1, inplace=True)
data_2.drop('id', axis=1, inplace=True)

<div class='alert alert-success'> ✔️ Отмечу, что колонка id, с одной стороны, как ты пишешь, является неинформативным для модели идентификатором, с другой содержит дубли, что в других задачах могло стать проблемой. 
</div>

Проверим данные на наличие дубликатов.

In [18]:
print("Количество дубликатов в data_0:", data_0.duplicated().sum())
print("Количество дубликатов в data_1:", data_1.duplicated().sum())
print("Количество дубликатов в data_2:", data_2.duplicated().sum())

Количество дубликатов в data_0: 0
Количество дубликатов в data_1: 0
Количество дубликатов в data_2: 0


Дубликаты отсутствуют.

Разделим данные на признаки и целевой признак:

In [19]:
features_0 = data_0.drop('product', axis=1)
target_0 = data_0['product']

features_1 = data_1.drop('product', axis=1)
target_1 = data_1['product']

features_2 = data_2.drop('product', axis=1)
target_2 = data_2['product']

<div class='alert alert-warning'>⚠️
Хорошо, но правильнее fit scaler-а делать на train выборке(<a href='https://dzen.ru/media/id/5ee6f73b7cadb75a66e4c7e3/kak-sdelat-preprocessing-nadejnee-i-sokratit-kod-v-tri-raza-640dbee70a7c6b686e54bf27'>как здесь</a>), так как модель о распределении параметров точек из других выборок на этапе обучения может не знать. Я бы в функцию ниже его советовал занести


</div>

<div class="alert alert-info">
Согласна с замечанием, масштабирование признаков должно происходить на обучающей выборке.
</div>

<div class='alert alert-success'> ✔️(2)👍
</div>

Теперь у нас подготовлены данные для каждого региона, которые можно использовать для обучения и валидации модели линейной регрессии.

## Обучение и проверка модели

Обучим и проверим модель для каждого региона:

1. Разобьём данные на обучающую и валидационную выборки в соотношении 75:25.
2. Обучим модель и сделаем предсказания на валидационной выборке.
3. Сохраним предсказания и правильные ответы на валидационной выборке.
4. Выведем на экран средний запас предсказанного сырья и RMSE модели.

Напишем функцию, объединяющую обучение модели и вывод результатов:

In [20]:
def train_and_evaluate_model(features, target):
    # Разбиваем данные на обучающую и валидационную выборки (75:25)
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=42)
    
    # Масштабируем признаки
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Обучаем модель линейной регрессии
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    # Делаем предсказания на валидационной выборке
    y_pred = model.predict(X_valid_scaled)

    # Считаем метрики: средний запас предсказанного сырья и RMSE модели
    mean_product = y_pred.mean()
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))

    # Возвращаем результаты
    return y_pred, y_valid, mean_product, rmse, y_valid.reset_index(drop=True), pd.Series(y_pred)


<div class='alert alert-success'> ✔️Молодец, что однотипные действия (разбиение на выборки, построение регрессии, подсчет ошибки) объединяешь в функции, чтобы избежать описок и сделать код компактнее.
</div>

<div class="alert alert-info">
Теперь масштабирование признаков происходит на обучающей выборке, и затем преобразование применяется к валидационной выборке. Это предотвратит утечку информации между выборками и позволит нам получить более корректные результаты.
</div>

Обучаем и оцениваем модель для каждого региона.

In [21]:
# Создаем список с наборами масштабированных признаков и целевых признаков для каждого региона
features_list = [features_0, features_1, features_2]
target_list = [target_0, target_1, target_2]

true_values_list = []
predictions_list = []

for i, (features, target) in enumerate(zip(features_list, target_list)):
    y_pred, y_valid, mean_product, rmse, true_values, predictions = train_and_evaluate_model(features, target)
    print(f"Регион {i}:")
    print(f"Средний запас предсказанного сырья: {mean_product:.2f}")
    print(f"RMSE модели: {rmse:.2f}")
    print()
    # Сохраняем истинные значения и предсказания для каждого региона
    true_values_list.append(true_values)
    predictions_list.append(predictions)

Регион 0:
Средний запас предсказанного сырья: 92.40
RMSE модели: 37.76

Регион 1:
Средний запас предсказанного сырья: 68.71
RMSE модели: 0.89

Регион 2:
Средний запас предсказанного сырья: 94.77
RMSE модели: 40.15



На основе полученных результатов можно сделать следующие выводы:

- Регион 0: этот регион имеет средний запас сырья ниже, чем регион 2, но выше, чем регион 1. Однако, RMSE модели в этом регионе высокий, что указывает на большую ошибку в предсказаниях.
- Регион 1: в этом регионе средний запас сырья ниже, чем в остальных регионах. Однако, RMSE модели очень низкий, что говорит о высокой точности предсказаний.
- Регион 2: имеет наибольший средний запас сырья среди всех регионов, но также имеет самую высокую ошибку в предсказаниях (RMSE).

<div class='alert alert-success'> ✔️Отлично, построила регрессию, посчитала метрики, правильно отметила в выводах, что в одном регионе меньше запасов, чем в двух других, зато ошибка тоже меньше. Тут бы я задачи максимизации среднего запаса не разделял от ошибки, поэтому пока конкретный регион лучше не рекомендовать
</div>

<div class="alert alert-info">
Вывод скорректирован.
</div>

<div class='alert alert-success'> ✔️(2)ОК
</div>

## Подготовка к расчёту прибыли

1. Все ключевые значения для расчётов сохраним в отдельных переменных.
2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.  
3. Сравним полученный объём сырья со средним запасом в каждом регионе.

Начнем с сохранения ключевых значений для расчетов в отдельных переменных:

In [22]:
# Количество исследуемых скважин
NUM_WELLS = 500

# Количество лучших скважин для разработки
NUM_BEST_WELLS = 200

# Бюджет на разработку скважин в регионе (в млн. рублей)
BUDGET = 10000000

# Доход с каждой единицы продукта (тыс. рублей)
INCOME_PER_UNIT = 450

# Бюджет на одну скважину (млн. рублей)
BUDGET_PER_WELL = BUDGET / NUM_BEST_WELLS

<div class='alert alert-success'> ✔️Хорошо. Рекомендовал бы задавать имена констант большими буквами, чтобы удобно было отличать их от переменных, это является соглашением по <a href='https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html'>PEP 8</a>.
</div>

<div class="alert alert-info">
Имена констант исправлены согласно руководству по стилю кода PEP 8.
</div>

<div class='alert alert-success'> ✔️(2)Точно.
</div>

Теперь рассчитаем достаточный объем сырья для безубыточной разработки новой скважины:

In [23]:
# Расчет объема сырья для безубыточной разработки (тыс. баррелей)
break_even_volume = BUDGET_PER_WELL / INCOME_PER_UNIT

<div class='alert alert-success'> ✔️Средний запас скважины для безубыточного бизнеса определен верно.
</div>

Сравним полученный объем сырья со средним запасом в каждом регионе:

In [24]:
# Средние запасы сырья по регионам (тыс.баррелей)
average_volumes = [predictions.mean() for predictions in predictions_list]

for i, avg_volume in enumerate(average_volumes):
    print(f"Регион {i}:")
    print(f"Средний запас сырья: {avg_volume:.2f} тыс. баррелей")
    print(f"Достаточный объем сырья для безубыточной разработки: {break_even_volume:.2f} тыс. баррелей")
    print(f"Разница между средним запасом и безубыточным объемом сырья: {avg_volume - break_even_volume:.2f} тыс. баррелей")
    print()

Регион 0:
Средний запас сырья: 92.40 тыс. баррелей
Достаточный объем сырья для безубыточной разработки: 111.11 тыс. баррелей
Разница между средним запасом и безубыточным объемом сырья: -18.71 тыс. баррелей

Регион 1:
Средний запас сырья: 68.71 тыс. баррелей
Достаточный объем сырья для безубыточной разработки: 111.11 тыс. баррелей
Разница между средним запасом и безубыточным объемом сырья: -42.40 тыс. баррелей

Регион 2:
Средний запас сырья: 94.77 тыс. баррелей
Достаточный объем сырья для безубыточной разработки: 111.11 тыс. баррелей
Разница между средним запасом и безубыточным объемом сырья: -16.34 тыс. баррелей



<div class='alert alert-success'> ✔️Молодец, что провела сравнение с безубыточной границей, лучше вместо констант средних запасов использовать переменные, чтобы код был гибче
</div>

<div class="alert alert-info">
Код исправлен, используются значения средних запасов, полученные после обучения модели.
</div>

<div class='alert alert-success'> ✔️(2)Отлично!
</div>

Исходя из полученных результатов, можем сделать следующие выводы:

1. Во всех трех регионах средний запас сырья в скважине меньше объема, необходимого для безубыточной разработки. Это означает, что простое среднее значение запасов сырья в скважинах каждого региона не обеспечивает безубыточность проекта.

2. Самый большой разрыв между средним запасом сырья и безубыточным объемом сырья наблюдается в регионе 1 (-42.40 тыс. баррелей). В регионе 0 и регионе 2 разница меньше: -18.71 тыс. баррелей и -16.34 тыс. баррелей соответственно.

Однако, стоит учесть, что эти результаты основаны на средних показателях и не учитывают потенциально более прибыльные отдельные скважины. Для выбора наиболее перспективных скважин и определения региона с наибольшей ожидаемой прибылью следует провести дополнительный анализ, применяя технику Bootstrap и оценивая риски.

<div class='alert alert-success'> ✔️Отличный вывод, значит случайный выбор точек приведет к убыткам, соответственно, важно подойти аккуратно к выбору скважин. 
</div>

## Расчёт прибыли и рисков 

1. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
2. Посчитаем риски и прибыль для каждого региона, применяя технику Bootstrap с 1000 выборок

Функция для расчета прибыли:

In [25]:
def profit(y_true, y_pred, top_n, price_per_barrel, budget):
    # Находим индексы скважин с максимальными предсказанными значениями
    top_indices = y_pred.argsort()[-top_n:]
    
    # Выбираем реальные значения объема сырья для этих скважин
    top_true_values = y_true.iloc[top_indices]
    
    # Рассчитываем доход от выбранных скважин
    revenue = top_true_values.sum() * price_per_barrel
    
    # Возвращаем прибыль, вычитая бюджет
    return revenue - budget

<div class='alert alert-success'> ✔️Функция profit корректная

</div>

Эта функция принимает на входе следующие аргументы:

- y_true: реальные значения запасов сырья в скважинах
- y_pred: предсказанные значения запасов сырья в скважинах
- top_n: количество лучших скважин для разработки
- price_per_barrel: доход с каждой единицы продукта
- budget: общий бюджет на разработку скважин

Функция возвращает значение прибыли, рассчитанное на основе выбранных лучших скважин для разработки.

Посчитаем риски и прибыль для каждого региона:

1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
2. Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
3. Напишем выводы: предложим регион для разработки скважин и обоснуем выбор.

Функция для подсчета результатов:

In [26]:
def bootstrap_analysis(y_true, y_pred, n_samples=1000, alpha=0.95, top_n=200, price_per_barrel=450, budget=10000000):
    profits = []

    for _ in range(n_samples):
        subsample = np.random.choice(y_true.index, size=500, replace=True)
        y_true_subsample = y_true.loc[subsample]
        y_pred_subsample = y_pred[subsample]
        profits.append(profit(y_true_subsample, y_pred_subsample, top_n, price_per_barrel, budget))

    profits = np.array(profits)
    mean_profit = profits.mean()
    confidence_interval = stats.t.interval(alpha, len(profits) - 1, loc=mean_profit, scale=profits.std())
    loss_risk = (profits < 0).mean()

    return mean_profit, confidence_interval, loss_risk

mean_profit_0, ci_0, risk_0 = bootstrap_analysis(true_values_list[0], predictions_list[0])
mean_profit_1, ci_1, risk_1 = bootstrap_analysis(true_values_list[1], predictions_list[1])
mean_profit_2, ci_2, risk_2 = bootstrap_analysis(true_values_list[2], predictions_list[2])

print("Регион 0: Средняя прибыль =", mean_profit_0, "95% доверительный интервал =", ci_0, "Риск убытков =", risk_0)
print("Регион 1: Средняя прибыль =", mean_profit_1, "95% доверительный интервал =", ci_1, "Риск убытков =", risk_1)
print("Регион 2: Средняя прибыль =", mean_profit_2, "95% доверительный интервал =", ci_2, "Риск убытков =", risk_2)

Регион 0: Средняя прибыль = 401775.415757689 95% доверительный интервал = (-113351.39608599216, 916902.2276013701) Риск убытков = 0.058
Регион 1: Средняя прибыль = 439744.9553116153 95% доверительный интервал = (37949.02465052891, 841540.8859727017) Риск убытков = 0.018
Регион 2: Средняя прибыль = 370964.69894932676 95% доверительный интервал = (-143543.45232071448, 885472.850219368) Риск убытков = 0.073


Исходя из проведенного анализа, рекомендуется выбрать регион 1 для разработки скважин. Обоснование этого выбора основано на следующих факторах:

1. Риск убытков в регионе 1 составляет всего 1.8%, что является наименьшим среди всех регионов и соответствует требованию риска убытков менее 2,5%.
2. Средняя прибыль в регионе 1 составляет 439 744 тыс. руб., что является наибольшим значением среди регионов, удовлетворяющих условию по риску убытков.

Таким образом, регион 1 является наиболее выгодным и безопасным вариантом для разработки скважин с точки зрения получения прибыли и минимизации риска убытков.

<div class='alert alert-success'> ✔️Отлично, провела бутстреп, вычислила целевые метрики
</div>

### Вывод:

При выборе региона для разработки нефтяных скважин важно учитывать не только потенциальную прибыль, но и риски, связанные с возможными убытками. В данном случае, проведя анализ данных и использовав технику бутстрэп, мы смогли оценить среднюю прибыль и риски для трех регионов.

Регион 1 оказался наиболее выгодным выбором, так как обеспечивает наименьший риск убытков и высокую среднюю прибыль. Инвестирование в этот регион позволит компании максимизировать свою прибыль при минимальных рисках.

<div class='alert alert-success'> 

✔️ Отличный выбор региона, трудоустроив тебя аналитиком, организация обязательно получила бы большую прибыль)
</div>

<div class='alert alert-info'>🏁 Отличная работа!  Подготовила признаки, корректно выполнила разделение на выборки и построила регрессию, посчитала целевые метрики и определила оптимальный регион с точки зрения прибыли и рисков убытков. Осталось скорректировать дать обратную связь по выделенным ячейкам.

</div>

<div class="alert alert-info">
Спасибо за комментарии, ты помог мне сделать мне проект еще лучше)
</div>

<div class='alert alert-info'> <strong>Ревьюер(2)</strong> <br>

Пожалуйста, спасибо за сильную работу! Удачи и интересных проектов впереди, покорения новых вершин в машинном обучении!👌


</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован